## Clustering

In [103]:
import numpy as np #판다스 친구 넘파이
import pandas as pd # 판다스
from pandas import DataFrame as df
import re # 정규식 사용을 위함
import nltk
import csv
from konlpy.tag import Okt
from konlpy.utils import pprint
from collections import Counter

In [115]:
import os
dataset = pd.read_csv("total_dataset_trim.csv",encoding='utf-8')
dataset

,NO,img_fold_no,studio_name,studio_picture,imafe_file,category,region,tag,price,area,floor,default_person,parking,description
0,1099,27,홍대_체스#4,['https://s3.hourplace.co.kr/temp/images/2019/...,"['27_0_홍대_체스#4.jpg', '27_1_홍대_체스#4.jpg', '27_2...",원룸,서울,#1.5룸#나혼자산다#모던#분리형원룸#빌라#서교동#서울시#원룸#클래식#홍대,"50,000",30,2,6,0,홍대 메인 거리에 등록한 오래된 집을 리모델링 했습니다. (90년대 빌라)\r\n체...
1,939,254,파티와 촬영용 이태원 뷰 좋은 루프탑 빌라,['https://s3.hourplace.co.kr/temp/images/2019/...,"['254_0_파티와_촬영용_이태원_뷰_좋은_루프탑_빌라.jpg', '254_1_파...",주택,서울,#10명수용#남산타워#루프탑#벽난로#복고풍인테리어#자연광스튜디오#전망,"38,000",145,2,8,1,"44평 내부에 큰 거실과 키친, 음악작업용 스튜디오 룸, 생활하는 큰 방, 렌트용 ..."
2,952,267,파티와 촬영용 이태원 뷰 좋은 루프탑 빌라,['https://s3.hourplace.co.kr/temp/images/2019/...,"['267_0_파티와_촬영용_이태원_뷰_좋은_루프탑_빌라.jpg', '267_1_파...",주택,서울,#10명수용#남산타워#루프탑#벽난로#복고풍인테리어#자연광스튜디오#전망,"38,000",145,2,8,1,"44평 내부에 큰 거실과 키친, 음악작업용 스튜디오 룸, 생활하는 큰 방, 렌트용 ..."
3,252,251,깔끔하고 아늑한 신축 투룸아파트,['https://s3.hourplace.co.kr/temp/images/2019/...,"['251_0_깔끔하고_아늑한_신축_투룸아파트.jpg', '251_1_깔끔하고_아늑...",아파트,서울,#10평투룸오피스텔#따뜻한가정집#선유도신축아파트#선유도신축오피스텔#신혼부부아파트#아...,"50,000",33,11,4,1,선유도역 1분거리에 위치한 신축 투룸오피스텔입니다.\r\n11층 남향으로 낮에는 해...
4,478,477,"TV없이 아늑하게 꾸민 거실, 21평 부부 공간",['https://s3.hourplace.co.kr/temp/images/2019/...,"['477_0_TV없이_아늑하게_꾸민_거실,_21평_부부_공간.jpg', '477_...",아파트,경기,#12층#TV없는#분위기좋은#색다른#아늑한거실#예쁜거실#깨끗한#쇼파2개#북향의#해가안드는,"62,000",69,12,10,3,- 2인가구 부부가 사는 공간입니다.\r\n- 신축이라 쾌적하고 깔끔합니다.\r\n...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1847,1804,89,홍대 2룸,['https://s3.hourplace.co.kr/image/user/841/20...,"['89_0_홍대_2룸.jpg', '89_1_홍대_2룸.jpg', '89_2_홍대_...",빌라,서울,NaN,"50,000",50,2,7,0,홍대에 있는 방 2개 주택입니다.
1848,1831,1,경리단길 윤싸롱 3호점 - 채광 굿 깔끔하고 모던한 자취방,['https://s3.hourplace.co.kr/image/user/6876/2...,"['1_0_경리단길_윤싸롱_3호점_-_채광_굿_깔끔하고_모던한_자취방.jpg', '...",원룸,서울,NaN,"40,000",33,1,3,1,낮에 채광 잘드는 1층 집이고 공간은 약 10평정도 됩니다. 드라마 이태원클라쓰 촬...
1849,1832,2,"상수역 6분거리 , 합정역 6분거리 30평급 넓은 공간 자연광 스튜디오 VILLA ...",['https://s3.hourplace.co.kr/image/user/6691/2...,"['2_0_상수역_6분거리_,_합정역_6분거리_30평급_넓은_공간_자연광_스튜디오_...",스튜디오,서울,NaN,"45,000",96,3,6,1,30평의 넓은 공간에 동남서 방향 3면에 창이 배치되어 자연광이 풍부하고 섹션별로 ...
1850,1840,10,시매,['https://s3.hourplace.co.kr/image/user/6818/2...,"['10_0_시매.jpg', '10_1_시매.jpg', '10_2_시매.jpg', ...",주택,서울,NaN,"41,000",46,4,3,2,해방촌 언저리에 있는 작은 옥탑방이예요.\r\n\r\n방문이 없는 투룸 과 주방 공...


In [135]:
# for i in range(6):
#     kkma = Kkma.nouns(
#     '홍대 메인 거리에 등록한 오래된 집을 리모델링 했습니다. (90년대 빌라)\r\n체스 그림을 바탕으로 모던 클래식한 분위기를 조성했습니다.\r\n주말(금/토)보다 평일(일~목)에 예약해주시면 감사하겠습니다.')
def getCoreTags(n):
    coreTagList=[]
    kkma=Kkma()    
    for i in range(n):
        coreTag=[]
        desc=kkma.nouns(dataset.description[i])
        print("desc : ",desc)
        tags=(dataset.tag[i]).split('#')  # tag를 한번 더 파싱
        for t in tags:
            if t in desc:
                coreTag.append(t)
        if len(coreTag)>0:
            coreTagList.append(coreTag)
#         else:
#             coreTagList.append(tags)
    return coreTagList


In [136]:
%time getCoreTags(10)

desc :  ['홍', '홍대', '대', '메인', '거리', '등록', '집', '리모델링', '90', '90년대', '년대', '빌라', '체스', '그림', '바탕', '모던', '클래식', '분위기', '조성', '주말', '금', '토', '평일', '일', '목', '예약', '감사']


NameError: name 'tagList' is not defined

In [120]:
okt=Okt()
for j in range(len(dataset.description)):
    noun=okt.nouns(dataset.description[j])
    for i,n in enumerate(noun):
        if len(n)<2:
            noun.pop(i)
count=Counter(noun)

In [230]:
nounList=count.most_common(100)
for v in nounList:
    print(v)

('촬영', 3)
('자연', 2)
('상품', 2)
('감각', 2)
('구석', 2)
('용산', 1)
('위치', 1)
('복층', 1)
('오피스텔', 1)
('공간', 1)
('고가', 1)
('미터', 1)
('느낌', 1)
('남향', 1)
('통창', 1)
('콤보', 1)
('정도', 1)
('화장실', 1)
('제외', 1)
('모든', 1)
('사진', 1)
('보정', 1)
('층고', 1)
('화이트', 1)
('베이스', 1)
('인테리어', 1)
('바이럴', 1)
('미술', 1)
('전공자', 1)
('드라마', 1)
('화보', 1)


In [287]:
def getTagList(n):
    parseTagList=[]
    for i in range(n):
        tempTagList=[]
        tags=(dataset.tag[i]).split('#') # 태그별로 자르기
        for tag in tags:
            if len(tag)>1:
                kkma.nouns(tag)  # 1개 태그 내 복합 명사 자르기
                tempTagList.append(kkma.nouns(tag))
        parseTagList.append(tempTagList)
    return parseTagList

In [288]:
getTagList(3)

[[['1.5', '1.5룸', '룸'],
  ['나', '혼자'],
  ['모던'],
  ['분리', '분리형원룸', '형', '원룸'],
  ['빌라'],
  ['서교', '서교동', '동'],
  ['서울시'],
  ['원룸'],
  ['클래식'],
  []],
 [['10', '10명수용', '명', '수용'],
  ['남산', '남산타워', '타워'],
  ['루프', '루프탑', '탑'],
  ['벽난로'],
  ['복고', '복고풍인테리어', '풍인', '테리어'],
  ['자연광', '자연광스튜디오', '스튜디오'],
  ['전망']],
 [['10', '10명수용', '명', '수용'],
  ['남산', '남산타워', '타워'],
  ['루프', '루프탑', '탑'],
  ['벽난로'],
  ['복고', '복고풍인테리어', '풍인', '테리어'],
  ['자연광', '자연광스튜디오', '스튜디오'],
  ['전망']]]

In [324]:
tagSet=[]
tagsList=[]
for j in range(len(getTagList(4))):
    for t in getTagList(4)[j]:
        print("t >>>>> ",t)
        tagList=[]
        for i in range(len(t)):
            if(len(t[i])>=2|len(t[i])<=7):
                print("t[i] : ",t[i])
                tagList.append(t[i])
        tagSet.append(tagList)


t >>>>>  ['1.5', '1.5룸', '룸']
t[i] :  1.5
t >>>>>  ['나', '혼자']
t[i] :  혼자
t >>>>>  ['모던']
t[i] :  모던
t >>>>>  ['분리', '분리형원룸', '형', '원룸']
t[i] :  분리
t[i] :  원룸
t >>>>>  ['빌라']
t[i] :  빌라
t >>>>>  ['서교', '서교동', '동']
t[i] :  서교
t[i] :  서교동
t >>>>>  ['서울시']
t[i] :  서울시
t >>>>>  ['원룸']
t[i] :  원룸
t >>>>>  ['클래식']
t[i] :  클래식
t >>>>>  []
t >>>>>  ['10', '10명수용', '명', '수용']
t[i] :  10
t[i] :  수용
t >>>>>  ['남산', '남산타워', '타워']
t[i] :  남산
t[i] :  타워
t >>>>>  ['루프', '루프탑', '탑']
t[i] :  루프
t[i] :  루프탑
t >>>>>  ['벽난로']
t[i] :  벽난로
t >>>>>  ['복고', '복고풍인테리어', '풍인', '테리어']
t[i] :  복고
t[i] :  복고풍인테리어
t[i] :  풍인
t[i] :  테리어
t >>>>>  ['자연광', '자연광스튜디오', '스튜디오']
t[i] :  자연광
t[i] :  자연광스튜디오
t >>>>>  ['전망']
t[i] :  전망
t >>>>>  ['10', '10명수용', '명', '수용']
t[i] :  10
t[i] :  수용
t >>>>>  ['남산', '남산타워', '타워']
t[i] :  남산
t[i] :  타워
t >>>>>  ['루프', '루프탑', '탑']
t[i] :  루프
t[i] :  루프탑
t >>>>>  ['벽난로']
t[i] :  벽난로
t >>>>>  ['복고', '복고풍인테리어', '풍인', '테리어']
t[i] :  복고
t[i] :  복고풍인테리어
t[i] :  풍인
t[i] :  테리어
t >>>>>  ['자연광'

In [297]:
print("tagSet : ",tagSet)

tagSet :  [['1.5', '1.5룸'], ['혼자'], ['모던'], ['분리', '분리형원룸', '원룸'], ['빌라'], ['서교', '서교동'], ['서울시'], ['원룸'], ['클래식'], []]
